In [2]:
import numpy as np
import pandas as pd
from helper import Gini, features_type
from scipy import sparse
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#for NN model
from keras.layers.advanced_activations import PReLU
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.callbacks import History 
from time import time
import datetime
from sklearn.model_selection import StratifiedKFold
from collections import defaultdict

Using TensorFlow backend.


# 1. Load Data #

- Create train, test dataset
- Create train target label
- Create feature: miss, interval, ordinal, binary, nominal


In [3]:
cv_only = True
save_cv = True


#read data
train = pd.read_csv("./input/train.csv")
train_label = train['target']
train_id = train['id']
del train['target'], train['id']

test = pd.read_csv("./input/test.csv")
test_id = test['id']
del test['id']

#find missing value by each row and recode to column 'missing'
train['missing'] = (train==-1).sum(axis=1).astype(float)
test['missing'] = (test==-1).sum(axis=1).astype(float)

# extract interval, ordinal, binary and nominal feature
interval_fea, ordinal_fea, binary_fea, nominal_fea = features_type(train)

# 2. Feature Engineering #

## 2.1 Feature importance

In [4]:
from numpy import loadtxt
from xgboost import XGBClassifier, plot_importance
from matplotlib import pyplot
FImodel = XGBClassifier()
FImodel.fit(train, train_label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [5]:
#plot
# from xgboost import plot_importance
# plot_importance(FImodel, max_num_features = 15)
# pyplot.show()

#list the most important feature with descending
sorted_idx = np.argsort(FImodel.feature_importances_)[::-1]
# for index in sorted_idx:
#     print([train.columns[index], FImodel.feature_importances_[index]])

#extract first 15 most importacne features
impo_fea=train.columns[sorted_idx[:15]]

## 2.1 Nominal Data Encoding ##

In [6]:
# Nomianl  feature with one hot encoding
for c in nominal_fea:
    le = LabelEncoder()
    x = le.fit_transform(pd.concat([train[nominal_fea], test[nominal_fea]])[c])
    train[c] = le.transform(train[c])
    test[c] = le.transform(test[c])
    
enc = OneHotEncoder()
enc.fit(pd.concat([train[nominal_fea], test[nominal_fea]]))
train_nominal = enc.transform(train[nominal_fea])
test_nominal = enc.transform(test[nominal_fea])

## 2.2 Ordinal, binary, interval data ##

In [7]:
train_ordinal = train[ordinal_fea]
test_ordinal = test[ordinal_fea]
train_bin = train[binary_fea]
test_bin =test[binary_fea]
train_intv= train[interval_fea]
test_intv=test[interval_fea]
train_miss = train['missing'].values.reshape(-1,1)
test_miss=test['missing'].values.reshape(-1,1)

## 2.3 target_encoding ##


In [8]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [9]:
target_encoding_columns = list(interval_fea) + list(nominal_fea)
for f in target_encoding_columns:
    train[f + "_tef"], test[f + "_tef"] = target_encode(trn_series=train[f],
                                         tst_series=test[f],
                                         target=train_label,
                                         min_samples_leaf=200,
                                         smoothing=10,
                                         noise_level=0)
train_tef = train[[x for x in train.columns if 'tef' in x]]
test_tef = test[[x for x in train.columns if 'tef' in x]]

## 2.4 Combine all feature together, and get ready for training ##

In [10]:
'''
Building a training and testing list
'''
train_list = [train_nominal, train_ordinal, train_bin, train_intv, train_miss, train_tef] 
test_list = [test_nominal , test_ordinal, test_bin, test_intv, test_miss, test_tef]

In [11]:
'''
stack with sparse matrix and convert to array
'''
X = sparse.hstack(train_list).tocsr().toarray()
tem_test = sparse.hstack(test_list).tocsr().toarray()

In [12]:
X_test=[]
train_impo= train[impo_fea]
test_impo= test[impo_fea]
for i in range(train_impo.shape[1]):
    X_test.append(test_impo.values[:,i].reshape(-1,1))
X_test.append(tem_test)

# 3. Training NN Model with Keras # 

## 3.1 Build the model ##

In [15]:
def nn_model():
    inputs = []
    layers = []
    for _ in impo_fea:
        input_impo = Input(shape=(1, ), dtype='float32')
        net_impo = Dense(64, kernel_initializer='he_normal')(input_impo)
        net_impo = PReLU()(net_impo)
        net_impo = BatchNormalization()(net_impo)
        net_impo = Dropout(0.25)(net_impo)
        inputs.append(input_impo)
        layers.append(net_impo)

    input_num = Input(shape=(X.shape[1],), dtype='float32')
    layers.append(input_num)
    inputs.append(input_num)

    flatten = Concatenate(axis=-1)(layers)

    fc1 = Dense(512, kernel_initializer='he_normal')(flatten)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.5)(fc1)
    fc1 = Dense(64, kernel_initializer='he_normal')(fc1)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.25)(fc1)

    outputs = Dense(1, kernel_initializer='he_normal', activation='sigmoid')(fc1)
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return (model)

## 3.2 Start to Train ##

In [23]:
"""
NFOLDS = 5
"""
cv_train = np.zeros(len(train_label))
cv_pred = np.zeros(len(test_id))

#validation fold
NFOLDS = 2
kfold = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=218)

begintime = time()
history = History()
loss_record= defaultdict(list)
Gini_record={}
Gini_record['Gini']=[]
n=1
for (train_index, valid_index) in kfold.split(X, train_label):
    x_train = X[train_index]
    y_train = train_label[train_index]
    x_valid = X[valid_index]
    y_valid = train_label[valid_index]

    x_train_impo = train_impo.values[train_index]
    x_valid_impo = train_impo.values[valid_index]


   # get x_train x_test cat
    train_list, valid_list = [], []
    for i in range(x_train_impo.shape[1]):
        train_list.append(x_train_impo[:, i].reshape(-1, 1))
        valid_list.append(x_valid_impo[:, i].reshape(-1, 1))

    train_list.append(x_train)
    valid_list.append(x_valid)
    model = nn_model()

    hist = model.fit(train_list, y_train, epochs=2, batch_size=512, verbose=2, validation_data=[valid_list, y_valid], callbacks=[history])
    cv_train[valid_index] += model.predict(x=valid_list, batch_size=512, verbose=0)[:,0]
    Gini_scores= Gini(train_label[valid_index], cv_train[valid_index])
#     print('Current Folder Scores: ',Gini_scores)
    cv_pred += model.predict(x=X_test, batch_size=512, verbose=0)[:,0]
    
    
    #loss record
    loss_name='loss_'+ str(n)
    val_loss_name='val_loss_'+str(n)
    loss_record[loss_name]=hist.history['loss']
    loss_record[val_loss_name]=hist.history['val_loss']
    n+=1
    
    #Gini Record
    Gini_record['Gini'].append(Gini_scores)
    
    

print(Gini(train_label, cv_train ))
print("Total training time: ",str(datetime.timedelta(seconds=time() - begintime)))

# pd.DataFrame({'id': test_id, 'target': cv_pred * 1./ (NFOLDS)}).to_csv('../model/nnModelpred.csv', index=False)
pd.DataFrame(loss_record).to_csv('../model/loss_Record.csv', index=False)
pd.DataFrame(Gini_Record).to_csv('../model/Gini_Record.csv', index=False)




Train on 297606 samples, validate on 297606 samples
Epoch 1/2
 - 65s - loss: 0.2690 - val_loss: 0.1548
Epoch 2/2
 - 60s - loss: 0.1612 - val_loss: 0.1540
Train on 297606 samples, validate on 297606 samples
Epoch 1/2
 - 72s - loss: 0.2695 - val_loss: 0.1602
Epoch 2/2
 - 66s - loss: 0.1639 - val_loss: 0.1547
0.226818691295
Total training time:  0:06:35.754765


In [54]:
prediction= model.predict(x=X_test, batch_size=512, verbose=0)

In [27]:
pd.DataFrame(loss_record).to_csv('./model/loss_Record.csv', index=False)
pd.DataFrame(Gini_record).to_csv('./model/Gini_Record.csv', index=False)


In [20]:
hist.history.keys()

dict_keys(['val_loss', 'loss'])

In [55]:
print(prediction.shape)

(892816, 1)


In [71]:
for layer in nn_model.layers:
    print(layer.get_output_at(0).get_shape().as_list())

AttributeError: 'function' object has no attribute 'layers'